In [1]:
# 감성분석

In [2]:
import glob
from afinn import Afinn

pos_review = (glob.glob("/Users/ian/Desktop/Study/data/imdb/train/pos/*.txt"))[20]

f = open(pos_review, 'r')
lines1 = f.readlines()[0]
f.close()

afinn = Afinn()
afinn.score(lines1)

17.0

In [3]:
files = list(glob.glob("/Users/ian/Desktop/Study/data/imdb/train/pos/*.txt")[:10])

afinn = Afinn()

for i in files:
    f = open(i)
    lines1 = f.readlines()[0]
    print(afinn.score(lines1))
    f.close()

7.0
9.0
15.0
9.0
14.0
14.0
23.0
8.0
17.0
4.0


In [4]:
neg_review = (glob.glob("/Users/ian/Desktop/Study/data/imdb/train/neg/*.txt"))[20]

f = open(neg_review, 'r')
lines2 = f.readlines()[0]
f.close()

afinn.score(lines2)

2.0

In [5]:
# 기계학습으로 감성분석
# 로지스틱 회귀분석부터

In [6]:
pos_review = (glob.glob("/Users/ian/Desktop/Study/data/imdb/train/pos/*.txt")[:100])

lines_pos = []
for i in pos_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except Exception as e:
        continue

len(lines_pos)

100

In [7]:
neg_review = (glob.glob("/Users/ian/Desktop/Study/data/imdb/train/neg/*.txt")[:100])

lines_neg = []
for i in neg_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_neg.append(temp)
        f.close()
    except Exception as e:
        continue

len(lines_neg)

100

In [8]:
total_text = lines_pos + lines_neg
len(total_text)

200

In [9]:
import numpy as np
from nltk.corpus import stopwords

x = np.array(["pos","neg"])
class_Index = np.repeat(x,[len(lines_pos),len(lines_neg)],axis=0)
stop_words = stopwords.words('english')

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vect = TfidfVectorizer(stop_words=stop_words).fit(total_text)
X_train_vectorized = vect.transform(total_text)
X_train_vectorized.index = class_Index

df = pd.DataFrame(X_train_vectorized.toarray(), columns=vect.vocabulary_.keys())
df.head()

,movie,gets,respect,sure,lot,memorable,quotes,listed,gem,imagine,...,declared,cardiac,induced,performing,autopsies,qualifications,frighteningly,conclude,residency,wicker
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.029184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(random_state=10)
logit.fit(X_train_vectorized, class_Index)

LogisticRegression(random_state=10)

In [12]:
def pos_review(model):
    count_all = 0
    count = 0
    num = 100
    tests1 = []
    for idx in range(0,num):
        pos_review_test = (glob.glob("/Users/ian/Desktop/Study/data/imdb/test/pos/*.txt"))[idx]
        f = open(pos_review_test, 'r', encoding='utf-8')
        tests1.append(f.readlines())
        f.close()
    for test in tests1:
        preds = model.predict(vect.transform(test))
        result = preds[0]
        if result == "pos":
            count += 1
        count_all += 1
    rate = count*100/count_all
    print("예측정확도:{:.1f}%".format(rate))

In [13]:
def neg_review(model):
    count_all = 0
    count = 0
    num = 100
    tests2 = []
    for idx in range(0,num):
        neg_review_test = (glob.glob("/Users/ian/Desktop/Study/data/imdb/test/neg/*.txt"))[idx]
        f = open(neg_review_test, 'r', encoding='utf-8')
        tests2.append(f.readlines())
        f.close()
    for test in tests2:
        preds = model.predict(vect.transform(test))
        result = preds[0]
        if result == "neg":
            count += 1
        count_all += 1
    rate = count*100/count_all
    print("예측정확도:{:.1f}%".format(rate))

In [14]:
pos_review(logit)
neg_review(logit)

예측정확도:65.0%
예측정확도:70.0%


In [15]:
# 의사결정나무

In [16]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=10)
tree.fit(X_train_vectorized, class_Index)

pos_review(tree)
neg_review(tree)

예측정확도:62.0%
예측정확도:65.0%


In [17]:
# 랜덤포레스트

In [18]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=10, random_state=10)
forest.fit(X_train_vectorized, class_Index)

pos_review(forest)
neg_review(forest)

예측정확도:40.0%
예측정확도:81.0%


In [19]:
# 최근접이웃knn

In [20]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train_vectorized, class_Index)

pos_review(knn)
neg_review(knn)

예측정확도:22.0%
예측정확도:95.0%


In [21]:
# 인공신경망

In [22]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=10)
mlp.fit(X_train_vectorized, class_Index)

pos_review(mlp)
neg_review(mlp)

예측정확도:77.0%
예측정확도:71.0%


In [23]:
from sklearn.svm import SVC

svm = SVC(random_state=10)
svm.fit(X_train_vectorized, class_Index)

pos_review(svm)
neg_review(svm)

예측정확도:68.0%
예측정확도:66.0%


In [ ]:
# TextBlob

In [25]:
from textblob import TextBlob

In [26]:
a = TextBlob("파이썬은 재미있다.")
a.detect_language()

'ko'

In [27]:
b = TextBlob("I like Python,")
b.detect_language()

'en'

In [28]:
c = TextBlob("파이썬은 재미있다.")
c.translate(to="en")

TextBlob("Python is fun.")

In [29]:
from textblob.classifiers import NaiveBayesClassifier

train = [
    ('I love this sandwich', 'pos'),
    ('This is an amazing place', 'pos'),
    ('I feel very good about it', 'pos'),
    ('This is my best work', 'pos'),
    ('Whan an awesome view', 'pos'),
    ('I do not like this restaurant', 'neg'),
    ('I am tired of this stuff', 'neg'),
    ('I cannot deal with this', 'neg'),
    ('He is my nemesis', 'neg'),
    ('My obss is horrible', 'neg')
]

test = [
    ('The beer was good', 'pos'),
    ('I do not enjoy my job', 'neg'),
    ('I do not feel great today', 'neg'),
    ('This is amazing', 'pos'),
    ('Gary is a good friend of mine', 'pos'),
    ('I cannot believe I am doing this', 'neg')
]

cl = NaiveBayesClassifier(train)

In [32]:
print(cl.classify("This burgers are amazing"))
print(cl.classify("I cannot enjoy this pizza"))

pos
neg


In [33]:
blob = TextBlob("The beer was amazing. But the hangover was horrible. My boss was not happy", classifier=cl)
blob.classify()

'neg'

In [34]:
for sentence in blob.sentences:
    print(sentence, '==>', sentence.classify())

The beer was amazing. ==> pos
But the hangover was horrible. ==> neg
My boss was not happy ==> neg


In [35]:
for row in test:
    print(row[0], '==>', cl.classify(row[0]))

The beer was good ==> pos
I do not enjoy my job ==> neg
I do not feel great today ==> neg
This is amazing ==> pos
Gary is a good friend of mine ==> pos
I cannot believe I am doing this ==> neg


In [36]:
cl.accuracy(test)

1.0

In [38]:
cl.show_informative_features(10)

Most Informative Features
          contains(this) = True              neg : pos    =      2.3 : 1.0
          contains(this) = False             pos : neg    =      1.8 : 1.0
          contains(This) = False             neg : pos    =      1.6 : 1.0
            contains(an) = False             neg : pos    =      1.6 : 1.0
           contains(not) = False             pos : neg    =      1.6 : 1.0
             contains(I) = False             pos : neg    =      1.4 : 1.0
             contains(I) = True              neg : pos    =      1.4 : 1.0
            contains(He) = False             pos : neg    =      1.2 : 1.0
            contains(My) = False             pos : neg    =      1.2 : 1.0
          contains(Whan) = False             neg : pos    =      1.2 : 1.0
